In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [2]:
# Load the housing dataset
data = pd.read_csv('/content/housing_data.csv', encoding='latin1', on_bad_lines='warn')

<ipython-input-2-2edbcd5f702a>:2: DtypeWarning: Columns (2,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/housing_data.csv', encoding='latin1', on_bad_lines='warn')


In [3]:
# Check for missing values
missing_values = data.isnull().sum()
print(missing_values)

# Separate numerical and categorical columns
numerical_cols = data.select_dtypes(include=['number']).columns
categorical_cols = data.select_dtypes(include=['object']).columns

# Impute missing values
data[numerical_cols] = data[numerical_cols].fillna(data[numerical_cols].mean())
data[categorical_cols] = data[categorical_cols].fillna(data[categorical_cols].mode().iloc[0])

# Check again for missing values to ensure imputation was successful
missing_values_after = data.isnull().sum()
print(missing_values_after)

Unnamed: 0                0
url                       0
id                        0
Lng                       1
Lat                       1
Cid                       1
tradeTime                 1
DOM                    4710
totalPrice                1
square                    1
livingRoom                1
drawingRoom               1
kitchen                   1
bathRoom                  1
floor                     1
constructionTime          1
renovationCondition       1
buildingStructure         1
ladderRatio               1
elevator                 22
subway                   22
district                  1
dtype: int64
Unnamed: 0             0
url                    0
id                     0
Lng                    0
Lat                    0
Cid                    0
tradeTime              0
DOM                    0
totalPrice             0
square                 0
livingRoom             0
drawingRoom            0
kitchen                0
bathRoom               0
floor                

In [4]:
# print column names
print(data.columns)

Index(['Unnamed: 0', 'url', 'id', 'Lng', 'Lat', 'Cid', 'tradeTime', 'DOM',
       'totalPrice', 'square', 'livingRoom', 'drawingRoom', 'kitchen',
       'bathRoom', 'floor', 'constructionTime', 'renovationCondition',
       'buildingStructure', 'ladderRatio', 'elevator', 'subway', 'district'],
      dtype='object')


In [5]:
# Encode categorical variables
le = LabelEncoder()
data['buildingStructure'] = le.fit_transform(data['buildingStructure'])

In [6]:
# Select relevant features
X = data.drop(['totalPrice'], axis=1)
y = data['totalPrice']

In [7]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
def calc_elements_upper_and_lower_than_IQR(X_train, district):
    try:
        X_train_square = X_train[district].astype(float)
        Q1 = np.percentile(X_train_square, 25, interpolation='midpoint')
        Q3 = np.percentile(X_train_square, 75, interpolation='midpoint')
        IQR = Q3 - Q1
        print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
        upper_element_mask = X_train_square >= (Q3 + 1.5 * IQR)
        lower_element_mask = X_train_square <= (Q1 - 1.5 * IQR)
        return upper_element_mask, lower_element_mask
    except ValueError as e:
        print(f"Error processing column {district}: {e}")
        return None, None

In [9]:
# Iterate over columns in X_train and check for non-numeric values
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        try:
            # Attempt to convert the column to numeric
            X_train[col] = pd.to_numeric(X_train[col], errors='raise')
        except ValueError:
            print(f"Column '{col}' contains non-numeric values.")
            # Handle the non-numeric values (e.g., drop the column, impute values, or use one-hot encoding)
            X_train = X_train.drop(col, axis=1)

Column 'url' contains non-numeric values.
Column 'id' contains non-numeric values.
Column 'tradeTime' contains non-numeric values.
Column 'livingRoom' contains non-numeric values.
Column 'drawingRoom' contains non-numeric values.
Column 'floor' contains non-numeric values.
Column 'constructionTime' contains non-numeric values.


In [10]:
# Polynomial Regression
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
poly_X_train = poly.fit_transform(X_train)
X_test = X_test[X_train.columns]  # Keep only columns present in X_train
poly_X_test = poly.transform(X_test)

In [11]:
# Linear Regression
linear_regression_model = LinearRegression()
linear_regression_model.fit(poly_X_train, y_train)

# Ridge Regression
ridge_regression_model = Ridge(alpha=2500.0)
ridge_regression_model.fit(poly_X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.85908e-55): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge(alpha=2500.0)

In [12]:
# Reduce the number of estimators
random_forest_model = RandomForestRegressor(n_estimators=10, random_state=42, n_jobs=-1)

# Fit the model
random_forest_model.fit(poly_X_train, y_train)


RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=42)

In [13]:
param_grid = {'alpha': [10.0, 100.0, 2500.0]}
grid_search = GridSearchCV(ridge_regression_model, param_grid, cv=5, scoring='r2')
grid_search.fit(poly_X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.1432e-57): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.56439e-57): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.41535e-57): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.40522e-57): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning

Best parameters: {'alpha': 2500.0}
Best score: -1147.0797972490375


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.85908e-55): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [14]:
y_pred = linear_regression_model.predict(poly_X_test)
print("Linear Regression R-squared score:", r2_score(y_test, y_pred))

y_pred = ridge_regression_model.predict(poly_X_test)
print("Ridge Regression R-squared score:", r2_score(y_test, y_pred))

y_pred = random_forest_model.predict(poly_X_test)
print("Random Forest Regressor R-squared score:", r2_score(y_test, y_pred))

Linear Regression R-squared score: -174034486.14335772
Ridge Regression R-squared score: -2.5080560360397606e+20
Random Forest Regressor R-squared score: 0.8956355055816707


In [15]:
y_pred = random_forest_model.predict(poly_X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 7630.013132698102


In [27]:
# Define parameter distribution for Randomized Search
param_dist = {
    'n_estimators': [50, 70, 5],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [32]:
from sklearn.model_selection import RandomizedSearchCV

# Limit the number of jobs to avoid excessive memory usage
random_search = RandomizedSearchCV(
    estimator=random_forest_model,
    param_distributions=param_dist,
    n_iter=5,  # Reduce number of iterations
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1,  # Reduce the number of parallel jobs
    scoring='neg_mean_squared_error'
)
random_search.fit(X_train, y_train)

# Best parameters from Randomized Search
print("Best parameters from Randomized Search: ", random_search.best_params_)

# Get the best estimator from the random search
best_estimator = random_search.best_estimator_
print("Best estimator from Randomized Search: ", best_estimator)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Best parameters from Randomized Search:  {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 20}
Best estimator from Randomized Search:  RandomForestRegressor(max_depth=20, min_samples_leaf=2, n_estimators=50,
                      n_jobs=-1, random_state=42)


In [34]:
# Predictions
y_pred_lr = linear_regression_model.predict(poly_X_test)
y_pred_rf = random_search.best_estimator_.predict(X_test)

In [35]:
# Evaluate Linear Regression
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
print(f"Linear Regression - MSE: {mse_lr}, R^2: {r2_lr}")

Linear Regression - MSE: 12723536197316.014, R^2: -174034486.14335772


In [36]:
# Evaluate Random Forest
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print(f"Random Forest - MSE: {mse_rf}, R^2: {r2_rf}")

Random Forest - MSE: 7072.286162657637, R^2: 0.9032641809508253


In [37]:

# Documentation for the model
documentation = """
# Home Value Prediction Model

## Model Architecture
The model uses a Random Forest Regressor optimized through Randomized Search for predicting home values.

## Training Process
1. Data loading and preprocessing
2. Handling missing values and encoding categorical variables
3. Feature selection
4. Splitting data into training and testing sets
5. Training regression models (Linear Regression and Random Forest)
6. Hyperparameter optimization using Grid Search and Randomized Search
7. Model evaluation using MSE and R^2

## Usage Instructions
1. Load your dataset into a Pandas DataFrame.
2. Preprocess the data (handle missing values, encode categorical variables).
3. Split the data into features (X) and target variable (y).
4. Split the data into training and testing sets.
5. Train the model using the provided script.
6. Evaluate the model performance.
7. Fine-tune and validate the model with new data.
"""

print(documentation)


# Home Value Prediction Model

## Model Architecture
The model uses a Random Forest Regressor optimized through Randomized Search for predicting home values.

## Training Process
1. Data loading and preprocessing
2. Handling missing values and encoding categorical variables
3. Feature selection
4. Splitting data into training and testing sets
5. Training regression models (Linear Regression and Random Forest)
6. Hyperparameter optimization using Grid Search and Randomized Search
7. Model evaluation using MSE and R^2

## Usage Instructions
1. Load your dataset into a Pandas DataFrame.
2. Preprocess the data (handle missing values, encode categorical variables).
3. Split the data into features (X) and target variable (y).
4. Split the data into training and testing sets.
5. Train the model using the provided script.
6. Evaluate the model performance.
7. Fine-tune and validate the model with new data.

